In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, dash_table, Input, Output
import dash_bootstrap_components as dbc


electricity = pd.read_csv("Electricity_European.csv")

min_year= electricity["year"].min()
max_year= electricity["year"].max()

app = Dash(external_stylesheets= [dbc.themes.SOLAR])

app.layout = html.Div([
    html.H1("Electricity Price by Europen"), 
    dcc.RangeSlider(id= "year_slider", 
                    min= min_year,
                    max= max_year, 
                    value= [min_year, max_year],
                    marks= {i: str(i) for i in range(min_year, max_year + 1)}), 
    dcc.Graph(id= "map_graph"), 
    dash_table.DataTable(id= "price-info", 
                         data= electricity.to_dict("records"))
])


@app.callback(
    Output("map_graph", "figure"), 
    Input("year_slider", "value")
)

def Update_Map_Graph(selected_years):
    filtered_electricity = electricity[
        (electricity["year"] >= selected_years[0]) & 
        (electricity["year"] <= selected_years[1])
    ]
    avg_price_electricity = filtered_electricity.groupby("Country")["Price (EUR/MWhe)"].mean().reset_index()

    map_fig = px.choropleth(avg_price_electricity,
                            locations= "Country", 
                            locationmode= "country names", 
                            color= "Price (EUR/MWhe)", 
                            scope= "europe", 
                            color_continuous_scale= "viridis")
    return map_fig


if __name__ == "__main__":
    app.run_server(debug= True)